In [1]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일 불러오기
OPENAI_KEY = os.getenv("OPENAI_KEY")

def should_reference_vector_db(QUERY):
    # OpenAI 클라이언트 초기화
    client = OpenAI(api_key=OPENAI_KEY)

    context = """
    Determine if the given query requires reference to the Vector DB.
    The Vector DB contains past conversations between the user and ChatGPT.
    Answer "YES" if referencing the Vector DB is necessary for the query to make sense,
    such as when the query refers to past interactions or continuity. Otherwise, answer "NO".

    [Examples]
    case 1: Query requiring Vector DB reference
    - Query: "나 오늘 마라탕 먹을거야!"
    - Explanation: The query implies a continuity in conversation and relies on past interactions to generate an appropriate response, such as, "너 어제도 먹었잖아!"
    - Output: "YES"

    case 2: Query requiring Vector DB reference
    - Query: "나 오늘 기분이 별로 안 좋아.."
    - Explanation: The query could use information from past conversations (e.g., the user likes strawberry cake) to generate a meaningful response, such as, "그럼 너가 좋아하는 딸기 케이크 먹어보는 거 어때?"
    - Output: "YES"

    case 3: Query not requiring Vector DB reference
    - Query: "오늘 날씨 진짜 좋다!"
    - Explanation: The query is about a general statement that does not relate to past interactions. Accessing the Vector DB is unnecessary.
    - Output: "NO"

    [Query]
    {query}
    """

    functions = [
        {
            "name": "determine_reference",
            "description": "Determine if referencing the Vector DB is necessary for the query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "answer": {
                        "type": "string",
                        "description": "The answer should be either 'YES' or 'NO'."
                    }
                },
                "required": ["answer"]
            }
        }
    ]

    try:
        # 쿼리 요청
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": context.format(query=QUERY)}
            ],
            tools=[{
                "type": "function",
                "function": functions[0]
            }],
            tool_choice={"type": "function", "function": {"name": "determine_reference"}}
        )

        # 응답 추출
        function_call = response.choices[0].message.tool_calls[0].function
        function_args = json.loads(function_call.arguments)

        answer = function_args['answer']
        return answer

    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return None


In [2]:
result = should_reference_vector_db("나 오늘 대학 친구들 만나러가!")
print(result)

NO
